DIGIT RECOGINITION

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tkinter as tk

In [10]:
# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [11]:
# Define the model
model = models.Sequential([
    layers.Dense(
        32, 
        activation='relu', 
        input_shape=(28 * 28,), 
        kernel_regularizer=regularizers.l2(0.002)),
    layers.Dense(
        32, 
        activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(
        10, 
        activation='softmax')
])

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(train_images, train_labels, epochs=20, batch_size=40, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6096 - loss: 1.2266 - val_accuracy: 0.9223 - val_loss: 0.3525
Epoch 2/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8620 - loss: 0.5604 - val_accuracy: 0.9412 - val_loss: 0.2819
Epoch 3/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8912 - loss: 0.4593 - val_accuracy: 0.9479 - val_loss: 0.2587
Epoch 4/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9019 - loss: 0.4101 - val_accuracy: 0.9503 - val_loss: 0.2419
Epoch 5/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9099 - loss: 0.3856 - val_accuracy: 0.9502 - val_loss: 0.2345
Epoch 6/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9134 - loss: 0.3635 - val_accuracy: 0.9492 - val_loss: 0.2436
Epoch 7/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9231 - loss: 0.3360 - val_accuracy: 0.9552 - val_loss: 0.2219
Epoch 8/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9226 - loss: 0.3341 - 

In [12]:
# Save the model after training
model.save("model_with_optimizer.h5")
print("Model saved successfully.")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc:.4f}')

Model saved successfully.
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9526 - loss: 0.2333
Test accuracy: 0.9584


In [13]:
import tkinter as tk  # Import tkinter with alias tk
from PIL import Image, ImageOps, ImageDraw
import numpy as np
import tensorflow as tf  # Import TensorFlow

In [14]:

# Load the saved model
model = tf.keras.models.load_model("D:\\Internship\\Zidio Internship\\Digit Recoginition\\model_with_optimizer.h5")

In [15]:
# Define preprocessing and prediction functions
def preprocess_image(image):
    image = image.resize((28, 28))  # Resize to 28x28 pixels
    image = image.convert('L')  # Convert to grayscale
    image = ImageOps.invert(image)  # Invert colors (white background, black digit)
    image = np.array(image).astype('float32') / 255  # Normalize pixel values
    image = image.reshape(1, 28 * 28)  # Flatten the image to match the model input
    return image

def predict_digit(image):
    processed_img = preprocess_image(image)
    prediction = model.predict(processed_img)
    return np.argmax(prediction)

In [16]:
# Tkinter GUI Application
class DrawAndPredictApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Draw a Digit")

        # Canvas for drawing
        self.canvas = tk.Canvas(master, width=200, height=200, bg="white")
        self.canvas.pack()

        # Buttons for prediction and clearing
        self.button_predict = tk.Button(master, text="Predict", command=self.make_prediction)
        self.button_predict.pack()

        self.button_clear = tk.Button(master, text="Clear", command=self.clear_canvas)
        self.button_clear.pack()

        # Bind mouse movement to paint function
        self.canvas.bind("<B1-Motion>", self.paint)

        # Create a blank image to draw on
        self.image = Image.new("L", (200, 200), 255)  # 200x200 white canvas
        self.draw = ImageDraw.Draw(self.image)

    def paint(self, event):
        # Draw a small circle at the mouse position
        x, y = event.x, event.y
        self.canvas.create_oval(x-5, y-5, x+5, y+5, fill="black", outline="black")
        self.draw.ellipse((x-5, y-5, x+5, y+5), fill="black", outline="black")

    def clear_canvas(self):
        # Clear the canvas and reset the image
        self.canvas.delete("all")
        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)

    def make_prediction(self):
        # Predict the digit and print it
        predicted_digit = predict_digit(self.image)
        print(f'Predicted Digit: {predicted_digit}')


# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawAndPredictApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Predicted Digit: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted Digit: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Digit: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Digit: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Digit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted Digit: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Digit: 8
